In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import optbinning as opt
%matplotlib inline
import pandas as pd
from sklearn.impute import SimpleImputer
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import OneHotEncoder, StandardScaler,KBinsDiscretizer, LabelEncoder
from sklearn.impute import SimpleImputer
import statsmodels.api as sm 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings("ignore")

(CVXPY) Feb 20 12:53:27 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Feb 20 12:53:27 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
data=pd.read_csv("../data/application_train_vf.csv",parse_dates=["date_mensuelle"], index_col=0)

In [3]:
variables_retenues=['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH'] #,'OWN_CAR_AGE'

In [4]:
data[variables_retenues].isna().mean()

CODE_GENDER                   0.000000
FLAG_OWN_CAR                  0.000000
FLAG_OWN_REALTY               0.000000
CNT_CHILDREN                  0.000000
AMT_INCOME_TOTAL              0.000000
AMT_CREDIT                    0.000000
AMT_ANNUITY                   0.000039
AMT_GOODS_PRICE               0.000905
NAME_INCOME_TYPE              0.000000
NAME_EDUCATION_TYPE           0.000000
NAME_FAMILY_STATUS            0.000000
NAME_HOUSING_TYPE             0.000000
REGION_POPULATION_RELATIVE    0.000000
DAYS_BIRTH                    0.000000
DAYS_EMPLOYED                 0.000000
DAYS_REGISTRATION             0.000000
DAYS_ID_PUBLISH               0.000000
dtype: float64

In [5]:
baseline_data=data[["TARGET"]+variables_retenues].dropna()

In [6]:
train_data, test_data=train_test_split(baseline_data, test_size=0.3, stratify=baseline_data["TARGET"], random_state=42)

In [7]:
formula =  "TARGET ~ CODE_GENDER+FLAG_OWN_CAR+FLAG_OWN_REALTY+CNT_CHILDREN+AMT_INCOME_TOTAL+AMT_CREDIT+AMT_ANNUITY+AMT_GOODS_PRICE+NAME_INCOME_TYPE+NAME_EDUCATION_TYPE+NAME_FAMILY_STATUS+NAME_HOUSING_TYPE+REGION_POPULATION_RELATIVE+DAYS_BIRTH+DAYS_EMPLOYED+DAYS_REGISTRATION+DAYS_ID_PUBLISH"

In [8]:
model_logit=sm.Logit.from_formula(formula=formula,data=train_data).fit()

         Current function value: 0.259103
         Iterations: 35


In [9]:
print(model_logit.summary())

                           Logit Regression Results                           
Dep. Variable:                 TARGET   No. Observations:               214072
Model:                          Logit   Df Residuals:                   214037
Method:                           MLE   Df Model:                           34
Date:                Tue, 20 Feb 2024   Pseudo R-squ.:                 0.04280
Time:                        12:53:40   Log-Likelihood:                -55467.
converged:                      False   LL-Null:                       -57947.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                           coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------
Intercept                                              -14.7985   1587.140     -0.009      0.993   -3125.536    3095.939
CODE_

In [10]:
y_pred_proba=model_logit.predict(test_data)

In [11]:
2*roc_auc_score(test_data["TARGET"], y_pred_proba) - 1

0.3229297846568788